In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


TASK 4:RECOMMENDATION SYSTEM
Create a simple recommendation system that suggests items to
users based on their preferences. You can use techniques like
collaborative filtering or content-based filtering to recommend
movies, books, or products to users.

In [ ]:
import pandas as pd

# Assuming the files are in the same directory as the notebook
# Replace '/content/drive/MyDrive/' with the correct path if needed

try:
    ratings = pd.read_csv('/content/drive/MyDrive/ratings.csv')
    print("ratings.csv loaded successfully.\n")
    print(ratings.head())
except FileNotFoundError:
    print("ratings.csv not found in the specified directory.")

try:
    links = pd.read_csv('/content/drive/MyDrive/links.csv')
    print("\nlinks.csv loaded successfully.\n")
    print(links.head())
except FileNotFoundError:
    print("links.csv not found in the specified directory.")

try:
    movies = pd.read_csv('/content/drive/MyDrive/movies.csv')
    print("\nmovies.csv loaded successfully.\n")
    print(movies.head())
except FileNotFoundError:
    print("movies.csv not found in the specified directory.")

try:
    tags = pd.read_csv('/content/drive/MyDrive/tags.csv')
    print("\ntags.csv loaded successfully.\n")
    print(tags.head())
except FileNotFoundError:
    print("tags.csv not found in the specified directory.")

ratings.csv loaded successfully.

   userId  movieId  rating  timestamp
0       1       17     4.0  944249077
1       1       25     1.0  944250228
2       1       29     2.0  943230976
3       1       30     5.0  944249077
4       1       32     5.0  943228858

links.csv loaded successfully.

   movieId  imdbId   tmdbId
0        1  114709    862.0
1        2  113497   8844.0
2        3  113228  15602.0
3        4  114885  31357.0
4        5  113041  11862.0

movies.csv loaded successfully.

   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3     

In [ ]:
!pip install pandas numpy scikit-surprise


**For getting different recommendations change user_id

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Load the datasets (using a smaller subset for demonstration)
ratings = pd.read_csv('/content/drive/MyDrive/ratings.csv', nrows=10000) # Load only 10,000 rows
movies = pd.read_csv('/content/drive/MyDrive/movies.csv')

# 1. Data Preprocessing:

# Create a user-item interaction matrix (ratings)
ratings_matrix = ratings.pivot_table(index='userId', columns='movieId', values='rating').fillna(0)

# 2. Collaborative Filtering:

# Calculate cosine similarity between users
user_similarity = cosine_similarity(ratings_matrix)

# 3. Making Recommendations:

def get_recommendations(user_id, similarity_matrix, ratings_matrix, num_recommendations=5):
    """
    Recommends movies for a given user based on similarity.

    Args:
        user_id: The ID of the user.
        similarity_matrix: The similarity matrix (user-user or item-item).
        ratings_matrix: The user-item interaction matrix.
        num_recommendations: The number of recommendations to generate.

    Returns:
        A list of recommended movie IDs.
    """
    # Check if user exists in matrix
    if user_id not in ratings_matrix.index:
        print(f"User {user_id} not found in ratings data.")
        return []

    user_index = ratings_matrix.index.get_loc(user_id)
    similar_users = similarity_matrix[user_index]

    # Weighted average of ratings for unrated movies
    predicted_ratings = np.dot(similar_users, ratings_matrix)

    # Exclude movies already rated by the user
    rated_movies = ratings_matrix.loc[user_id].iloc[ratings_matrix.loc[user_id].to_numpy().nonzero()[0]].index
    predicted_ratings = pd.Series(predicted_ratings, index=ratings_matrix.columns).drop(rated_movies)

    # Get top recommendations
    top_recommendations = predicted_ratings.sort_values(ascending=False).head(num_recommendations).index.tolist()

    return top_recommendations

# Example Usage:
user_id = 1
recommendations = get_recommendations(user_id, user_similarity, ratings_matrix)

# Print recommendations with movie titles:
if recommendations:
    recommended_movies = movies[movies['movieId'].isin(recommendations)]['title'].tolist()
    print(f"Recommended movies for user {user_id}: {recommended_movies}")

Recommended movies for user 1: ['Pulp Fiction (1994)', 'Shawshank Redemption, The (1994)', 'Godfather, The (1972)', 'Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)', 'American Beauty (1999)']
